In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-19"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10177,2024-05-19,Eua Nba,21:00,Denver Nuggets,Minnesota Timberwolves,1.50,2.72,196.5,1.81,2.04,-4.5,1.84,3.90,vJHjMg0m,0.666667,0.367647,0.552486,0.490196,0.034314,0.2,0.6,NaN,NaN,193.150,81.566845,0.422298,209.800,61.047273,0.290978,155.590,244.152,116.00,304.22,0.0,0.0,0.0,0.0,0.408848,0.084485,0.507540,-0.32,-0.064,-7.812500,0.735633,0.8,0.064367,2.20,0.440,3.909091,0.532684,0.8,0.267316
10178,2024-05-19,Alemanha Pro A,11:00,Frankfurt,Trier,1.94,1.78,151.5,1.80,1.86,1.5,1.81,1.64,rLFe7plJ,0.515464,0.561798,0.555556,0.537634,0.077262,0.4,0.6,NaN,NaN,130.904,25.013122,0.191080,133.234,53.360686,0.400504,111.624,147.490,144.76,80.01,0.0,0.0,0.0,0.0,0.060826,0.023184,0.069686,-1.05,-0.210,-4.476190,0.000000,0.0,0.000000,2.28,0.456,1.710526,0.000000,0.0,0.000000
10179,2024-05-19,Argentina Liga A,21:00,Union De Santa Fe,Obera TC,1.70,2.05,164.5,1.87,1.79,-2.5,1.94,2.50,llm143BS,0.588235,0.487805,0.534759,0.558659,0.076040,0.8,0.6,NaN,NaN,232.640,42.203604,0.181412,145.546,28.596999,0.196481,205.952,164.696,214.62,162.81,0.0,0.0,0.0,0.0,0.131993,0.030912,0.178369,1.22,0.244,2.868852,0.485391,0.4,-0.085391,-1.17,-0.234,-4.487179,0.430360,0.4,-0.030360
10180,2024-05-19,Austrália Nbl1 Leste,04:00,Inner West Bulls,Maitland M.,2.22,1.61,174.5,1.81,1.87,-2.5,1.97,1.78,8lY87w1l,0.450450,0.621118,0.552486,0.534759,0.071568,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.225240,0.023058,0.071653,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10181,2024-05-19,Austrália Nbl1 Norte,02:00,North Gold Coast Seahawks,South West Metro Pirates,1.48,2.53,179.5,1.81,1.87,-1.5,1.81,2.16,fq3ODIHp,0.675676,0.395257,0.552486,0.534759,0.070933,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.370305,0.023058,0.124679,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10182,2024-05-19,Austrália Nbl1 Norte,00:00,Rockhampton F,Northside Wizards F,3.20,1.32,151.5,1.82,1.88,1.5,1.85,1.58,UHTq8uxD,0.312500,0.757576,0.549451,0.531915,0.070076,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,185.30,0.0,0.0,0.0,0.0,0.588213,0.022933,0.111323,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10183,2024-05-19,Austrália Nbl1 Sul,01:00,Kilsyth,NW Tasmania,1.74,1.99,167.5,1.81,1.89,-1.5,1.84,2.10,4QpTZjLK,0.574713,0.502513,0.552486,0.529101,0.077225,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,155.40,147.03,0.0,0.0,0.0,0.0,0.094786,0.030578,0.093324,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10184,2024-05-19,Austrália Nbl1 Sul,01:30,Nunawading,Keilor Thunder,1.69,2.06,181.5,1.80,1.90,-3.5,2.02,1.98,bsk2Tlbl,0.591716,0.485437,0.555556,0.526316,0.077153,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,258.06,142.56,0.0,0.0,0.0,0.0,0.139536,0.038222,0.014142,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10185,2024-05-19,Filipinas Copa Das Filipinas,04:00,Ginebra Kings,Meralco Bolts,1.67,2.18,176.5,1.87,1.79,-3.5,1.98,2.71,Q9Qjr3w3,0.598802,0.458716,0.534759,0.558659,0.057518,0.0,0.0,NaN,NaN,NaN,NaN,NaN,173.354,26.735585,0.154225,0.000,0.000,147.90,159.08,0.0,0.0,0.0,0.0,0.187337,0.030912,0.220123,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10186,2024-05-19,França Lnb,14:00,Saint Quentin,Lyon-Villeurbanne,2.68,1.48,153.5,1.85,1.91,4.5,1.92,1.22,OCtXoJce,0.373134,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Denver Nuggets,Minnesota Timberwolves,196.5,1.81,1.0000,Over
1,21:00,Argentina Liga A,Union De Santa Fe,Obera TC,164.5,1.87,1.0000,Over
2,02:00,Austrália Nbl1 Norte,North Gold Coast Seahawks,South West Metro Pirates,179.5,1.81,0.8505,Over
3,01:00,Austrália Nbl1 Sul,Kilsyth,NW Tasmania,167.5,1.81,0.6581,Over
4,01:30,Austrália Nbl1 Sul,Nunawading,Keilor Thunder,181.5,1.80,0.9895,Over
5,04:00,Filipinas Copa Das Filipinas,Ginebra Kings,Meralco Bolts,176.5,1.87,1.0000,Over
6,15:00,França Pro B,ASA,La Rochelle,148.5,1.83,0.9998,Over
7,14:00,Israel Liga Leumit,Maccabi Rehovot,Gilboa Galil,149.5,1.81,0.9569,Over
8,13:00,Itália Série A2,Latina,Luiss Roma,160.5,1.80,0.9967,Over
9,15:30,Itália Série A2,Nardo,Cento,151.5,1.85,1.0000,Over
